<a href="https://colab.research.google.com/github/cnilsen/kcfp/blob/master/py-notebooks/maptoolv5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


King County Fish Passage Hydrology Tool 
---
---

## License
###### © 2020 Geosyntec Consultants Inc. The copyright holders grant the freedom to copy, modify, convey, adapt, and/or redistribute this work under the terms of the [Mozilla Public Licesnse 2.0](https://spdx.org/licenses/MPL-2.0.html)

## Overview
This tool supports King County's comprehensive inventory and assessment of assets on watercourses to
determine if the assets are barriers to fish passage. King County is performing the assessment using
methodology outlined in the Washington Department of Fish and Wildlife's Fish Passage Inventory,
Assessment, and Prioritization Manual (WDFW, 2019).
The Level B assessment outlined in the WDFW
2019 Manual provides for a hydraulic analysis to determine if a culvert meets the velocity and depth
requirements for fish passage between low and high fish passage flows. Per the WDFW 2019 Manual:
For an instream feature to be considered a non-barrier, it should not obstruct upstream migration
at any time between the low and high fish passage flows at that location:

* Low fish passage flow is the 95% exceedance flow; and
* High fish passage flow is the 10% exceedance flow during the months of adult upstream migration.

### Data Sources 

Data used in this tool has been provided from the Nature Conservancy's Stormwater Heatmap Project. Documentation on the Stormwater Heatmap is in progress. Below is a brief summary of data sources: 


* Digital Elevation Model - [USGS National Hydrography (NHD Plus High Resolution Raster Data)](https://www.usgs.gov/core-science-systems/ngp/national-hydrography/nhdplus-high-resolution)
* Soils - [USDA Gridded SURGO Database](https://www.nrcs.usda.gov/wps/portal/nrcs/detail/soils/home/?cid=nrcs142p2_053628)
* Slope - USGS National Elevation Dataset (https://ned.usgs.gov)
* Level B Sites - Spatial data provided by King County 
* Land cover and hydrology- The Nature Conservancy Stormwater Heatmap 

Documentation for the Stormwater Heatmap is in progress. Hydrology simulations were performed using [PyHSPF](https://github.com/djlampert/PyHSPF). Watershed factors used in this simulation were selected based on stormwater modeling guidance from the Department of Ecology. See the [Western Washington Hydrology Model Users Manual for more information.](https://fortress.wa.gov/ecy/madcap/wq/2014SWMMWWinteractive/2014%20SWMMWW.htm#Topics/VolumeIII2014/VolIII%20AppB%202014.htm%3FTocPath%3D2014%2520SWMMWW%7CVolume%2520III%2520-%2520Hydrologic%2520Analysis%2520and%2520Flow%2520Control%2520BMPs%7C)

### Limitations

##### Results not calibrated 
Hydrology simulations were performed on a hydrologic response unit basins using regional parameters. Parameters have not been calibrated to actual data. A model-based gridded precipitation dataset was used (see [Mauger et al 2018](https://cig.uw.edu/news-and-events/publications/new-projections-of-changing-heavy-precipitation-in-king-county/) ). 

##### Model does not include routing or storage 
Modeling was performed on a lumped parameter basis. Attenuation due to routing or storage is not reflected in the model. For large watersheds (greater than ~ 1 square mile) high flows may not be accurate. 

##### Model does not include deep groundwater baseflow
The modeling results include all hspf flow paths (surface flow, interflow, and groundwater flow), however baseflow from deep groundwater is not included. Consistent with WDFW guidance, results in this notebook have assumed a minimum baseflow of 1 cfs. 

---

## Version information 
alpha version - for evaluation. Released 15 Jan 2020

##### [Report a bug or other issue](mailto:incoming+geosyntec-king-county-fish-passage-tool-public-16324449-ai7zp34q4yd1im1aedhzmlalo-issue@incoming.gitlab.com)

---

## 1. Project set up
Import libraries and set paths

In [ ]:
from pathlib import Path
from datetime import datetime
from functools import partial

from google.oauth2 import service_account
import ee

import pandas
import pandas_gbq

import geopandas
from whitebox import WhiteboxTools

import ipywidgets
from ipywidgets import widgets
from IPython.display import display
import ipyleaflet
from ipyleaflet import GeoData, LayersControl, basemaps
from sidecar import Sidecar
import folium

import pikestreet


datadir = Path('..', '01-data').absolute()
outputdir = Path('..', '02-output', datetime.today().strftime('%Y%m%d')).absolute()
outputdir.mkdir(exist_ok=True, parents=True)

wbt = WhiteboxTools()
wbt.work_dir = str(Path('.').absolute())
wbt.verbose = False

%matplotlib inline

## 2. User Configuration 
#### Notebook variables
Edit the variables below for your particular case

In [ ]:
#Location of shapefile with analysis locaitons
sitefile = datadir / 'from_king_county' / 'levelBsites.geojson'
sites = geopandas.read_file(sitefile)

#Beginning year 
year0 = 1970 
#end year 
yearN = 2000 

#array of all months (1-12)
all_months = list(range(1,13))

#low flow is set for all months 
low_flow_months = all_months 

#high flow is set for upstream migration  
high_flow_months = list(range(1,2)) #high flow is calculated for January only 

numofQuantiles = 100 #number of quantitles to calculate

#### Earth Engine Authentication
This notebook requires a Google Earth Engine Account to query and analyze spatial data. Visit https://earthengine.google.com/signup to sign up for an account.  

After you have registered your account. Run the `ee.Authenticate` function to authenticate your access to Earth Engine servers and `ee.Initialize` to initialize it. Upon running the following cell you'll be asked to grant Earth Engine access to your Google account. Follow the instructions printed to the cell. (Sometimes this takes awhile). 

In [ ]:
#ee.Authenticate() #<---Uncomment and run the first time you need to authenticate your google earth account
ee.Initialize()

#### Local directory information

This notebook creates a local directory with today's date to store data. After you complete your analysis, you may want to delete the files in your local directory. 
The cell below outputs the working directory, the directory for input data, and the output directory. 

In [ ]:
print('Working directory:', wbt.work_dir)
print('Input data directory:', datadir)
print('Output data directory:', outputdir)

Working directory: C:\Users\cnilsen\Documents\repos\king-county-fish-passage\py-notebooks
Input data directory: C:\Users\cnilsen\Documents\repos\king-county-fish-passage\py-notebooks\..\01-data
Output data directory: C:\Users\cnilsen\Documents\repos\king-county-fish-passage\py-notebooks\..\02-output\20200303


## 3. Site Selection

Execute the cell below to display a map of locations to analyze. Once you have selected a point to analyze, select the 'Click to Analyze Site' button.

The notebook will then download a dem to your local directory and begin delineating the watershed. When the button turns green and check mark appears you can proceed to the next cell. 

In [ ]:
# Create the text boxes, etc. 
latitudeBox = widgets.FloatText(value=0, description='Lat:', disabled=True)
longitudeBox = widgets.FloatText(value=0, description='Lng:', disabled=True)
ID = widgets.Textarea(value='ID here', description='ID:', disabled=True)
       
point_layer = GeoData(geo_dataframe=sites, name='Level B Sites')
textarea = widgets.VBox([latitudeBox, longitudeBox, ID])
info_panel = widgets.Label(value='',  disabled=True, layout = {'visibility': 'hidden'})


watershed_dict= dict()
map1 = ipyleaflet.Map(
    zoom=9,
    center=(47.49169, -122.243516),
    layout=widgets.Layout(height='400px'),
    basemap=basemaps.Esri.WorldTopoMap
);


#Add level B Sites
accept_button = widgets.Button(
    description='Click to Analyze Site',
    disabled=True,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Analyze Site',
    layout = {'margin': '10px 10px 10px 10px'},
)

_point_handler = partial(pikestreet.onclick_handler, sites=sites, mapobj=map1,
                         lat_box=latitudeBox, lng_box=longitudeBox,
                         watershed_dict=watershed_dict, wbt=wbt,
                         accept_button=accept_button, id_box=ID)

_button_clicker = partial(pikestreet.accept_button_click_handler, mapobj=map1,
                          watershed_dict=watershed_dict, ee=ee, wbt=wbt,
                          outputdir=outputdir, info_panel=info_panel)

point_layer.on_click(_point_handler)
accept_button.on_click(_button_clicker)

map1.add_layer(point_layer)
map1.add_control(LayersControl(position='topright'))

col2 = widgets.VBox([accept_button, textarea, info_panel])
output_area = widgets.VBox([map1, col2])
with Sidecar(title='Watershed Delineator') as sc:
    display(output_area)

## 4. Land use, soils, and slope data 

Execute the cell below to get land use, soils and slope information for the watershed. 

In [ ]:
pikestreet.detail_map(watershed_dict, ee=ee)

|              |    |  | 
:-------------------------:|:-------------------------:|:-------------------------:
![alt text](legend_1.PNG "Title")  | ![alt text](legend_2.PNG "Title")| ![alt text](legend_0.PNG "Title") 



#### Get HRUs
These categories are combined to create "hydrologic response units" or HRUs. These are used to query the hydrology results. 

In [ ]:
HRUs = (
    pikestreet.get_HRUs_in_watershed(ee, watershed_dict['watershed_geometry'],
                                     image_file="users/stormwaterheatmap/public/hrus_2m")
        .pipe(pikestreet.process_HRUs)
)
pikestreet.hru_barchart(HRUs)

## 5. Get flow values from Big Query

#### Authenticate with service account credentials
a key file: `stormwaterheatmap-hydrology-8308154a1232.json` has been generated for use with this notebook. It is located in the data directory. Use this to create credintals to the Big Query api. 

In [ ]:
key_path = datadir / "stormwaterheatmap-hydrology-8308154a1232.json"
project_id = "stormwaterheatmap-hydrology"
pandas_gbq.context.project = project_id
pandas_gbq.context.credentials = service_account.Credentials.from_service_account_file(
    str(key_path)
)

centroid = watershed_dict['centroid'].iloc[0]
geohash = pikestreet.get_geohash_of_point(centroid, project_id=project_id)

low_flow_quants = pikestreet.get_flow_quantiles(HRUs, geohash, year0, yearN, numofQuantiles, *low_flow_months)
high_flow_quants = pikestreet.get_flow_quantiles(HRUs, geohash, year0, yearN, numofQuantiles, *high_flow_months)

quants = pikestreet.process_quantiles(low_flow_quants, 'Qlow').merge(
    pikestreet.process_quantiles(high_flow_quants, 'Qhigh'), on=['Exceedance']
)

#set a minimum 1 cfs flow
quants['Qlow'] = quants['Qlow'].clip(lower = 1)
quants['Qhigh'] = quants['Qhigh'].clip(lower = 1)

qplot = pikestreet.prob_plot(quants)

summary_table = quants.style.apply(
    lambda x: ["background: #1e88e5" if x['Exceedance']==0.10  else("background: #F58518" if x['Exceedance']==0.95 else "") for v in x], axis = 1).hide_index()
display(pikestreet.hilo_flow_widgets(quants),qplot.mark_line() + qplot.mark_point(),summary_table)